In [1]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
# matplotlib.use('Agg')
import datetime

%matplotlib inline
from finrl.meta.preprocessor.yahoodownloader import YahooDownloader
from finrl.meta.preprocessor.preprocessors import FeatureEngineer, data_split
from finrl.meta.env_stock_trading.env_stocktrading import StockTradingEnv
from finrl.agents.stablebaselines3.models import DRLAgent
from stable_baselines3.common.logger import configure
from finrl.meta.data_processor import DataProcessor

from finrl.plot import backtest_stats, backtest_plot, get_daily_return, get_baseline
from pprint import pprint

import sys
sys.path.append("../FinRL")

import itertools
from finrl import config
from finrl import config_tickers
import os
from finrl.main import check_and_make_directories
from finrl.config import (
    DATA_SAVE_DIR,
    TRAINED_MODEL_DIR,
    TENSORBOARD_LOG_DIR,
    RESULTS_DIR,
    INDICATORS,
    TRAIN_START_DATE,
    TRAIN_END_DATE,
    TEST_START_DATE,
    TEST_END_DATE,
    TRADE_START_DATE,
    TRADE_END_DATE,
)

/home/autolab/miniconda3/envs/fintech/lib/python3.10/site-packages/pyfolio/pos.py:26: UserWarning: Module "zipline.assets" not found; mutltipliers will not be applied to position notionals.
  warnings.warn(


In [2]:
check_and_make_directories([DATA_SAVE_DIR, TRAINED_MODEL_DIR, TENSORBOARD_LOG_DIR, RESULTS_DIR])

In [3]:
# 读取 ./StockData/stock_SH_SZ.csv

df = pd.read_csv('./StockData/stock_SH_SZ.csv')

# 将dt改名为date
df.rename(columns={'dt':'date'},inplace=True)
df.rename(columns={'kdcode':'tic'},inplace=True)

print(df.head())

         tic        date  close   open   high    low  prev_close  adjfactor  \
0  000001.SZ  2020-01-02  16.87  16.65  16.95  16.55       16.45  104.72765   
1  000001.SZ  2020-01-03  17.18  16.94  17.31  16.92       16.87  104.72765   
2  000001.SZ  2020-01-06  17.07  17.01  17.34  16.91       17.18  104.72765   
3  000001.SZ  2020-01-07  17.15  17.13  17.28  16.95       17.07  104.72765   
4  000001.SZ  2020-01-08  16.66  17.00  17.05  16.63       17.15  104.72765   

       turnover       volume  
0  2.571196e+09  153023187.0  
1  1.914495e+09  111619481.0  
2  1.477930e+09   86208350.0  
3  1.247047e+09   72860756.0  
4  1.423609e+09   84782412.0  


In [4]:
fe = FeatureEngineer(
                    use_technical_indicator=True,
                    tech_indicator_list = INDICATORS,
                    #use_vix=True,
                    use_turbulence=True,
                    user_defined_feature = False)

processed = fe.preprocess_data(df)

list_ticker = processed["tic"].unique().tolist()
list_date = list(pd.date_range(processed['date'].min(),processed['date'].max()).astype(str))
combination = list(itertools.product(list_date,list_ticker))

processed_full = pd.DataFrame(combination,columns=["date","tic"]).merge(processed,on=["date","tic"],how="left")
processed_full = processed_full[processed_full['date'].isin(processed['date'])]
processed_full = processed_full.sort_values(['date','tic'])

processed_full = processed_full.fillna(0)

Successfully added technical indicators
Successfully added turbulence index


In [5]:
TRAIN_START_DATE = "2020-01-01"
TRAIN_END_DATE   = "2022-12-31"
TRADE_START_DATE = "2023-01-01"
TRADE_END_DATE   = "2023-12-31"
train = data_split(processed_full, TRAIN_START_DATE,TRAIN_END_DATE)
trade = data_split(processed_full, TRADE_START_DATE,TRADE_END_DATE)
print(len(train))
print(train.head())
print(len(trade))
print(trade.head())

194376
         date        tic  close   open   high    low  prev_close   adjfactor  \
0  2020-01-02  000001.SZ  16.87  16.65  16.95  16.55       16.45  104.727650   
0  2020-01-02  000002.SZ  32.56  32.80  33.60  32.51       32.18  138.130321   
0  2020-01-02  000063.SZ  35.45  35.66  35.80  35.13       35.39   16.998256   
0  2020-01-02  000100.SZ   4.57   4.51   4.61   4.45        4.47    3.095800   
0  2020-01-02  000157.SZ   6.87   6.75   6.94   6.72        6.68   71.141680   

       turnover       volume  macd    boll_ub    boll_lb  rsi_30     cci_30  \
0  2.571196e+09  153023187.0   0.0  17.463406  16.586594   100.0  66.666667   
0  3.342374e+09  101213040.0   0.0  17.463406  16.586594   100.0  66.666667   
0  3.592989e+09  101425488.0   0.0  17.463406  16.586594   100.0  66.666667   
0  1.661807e+09  364974148.0   0.0  17.463406  16.586594   100.0  66.666667   
0  4.929516e+08   72029214.0   0.0  17.463406  16.586594   100.0  66.666667   

   dx_30  close_30_sma  close_60_sma 

In [6]:
stock_dimension = len(train.tic.unique())
state_space = 1 + 2*stock_dimension + len(INDICATORS)*stock_dimension
print(f"Stock Dimension: {stock_dimension}, State Space: {state_space}")
buy_cost_list = sell_cost_list = [0.001] * stock_dimension
num_stock_shares = [0] * stock_dimension

Stock Dimension: 267, State Space: 2671


In [7]:
env_kwargs = {
    "hmax": 100,
    "initial_amount": 1000000,
    "num_stock_shares": num_stock_shares,
    "buy_cost_pct": buy_cost_list,
    "sell_cost_pct": sell_cost_list,
    "state_space": state_space,
    "stock_dim": stock_dimension,
    "tech_indicator_list": INDICATORS,
    "action_space": stock_dimension,
    "reward_scaling": 1e-4
}

e_train_gym = StockTradingEnv(df = train, **env_kwargs)

env_train, _ = e_train_gym.get_sb_env()
print(type(env_train))

<class 'stable_baselines3.common.vec_env.dummy_vec_env.DummyVecEnv'>


In [8]:
agent = DRLAgent(env = env_train)

if_using_a2c = False
if_using_ddpg = False
if_using_ppo = False
if_using_td3 = False
if_using_sac = True

In [11]:

SAC_PARAMS = {
    "batch_size": 128,
    "buffer_size": 100000,
    "learning_rate": 0.0001,
    "learning_starts": 100,
    "ent_coef": "auto_0.1",
}

model_sac = agent.get_model("sac",model_kwargs = SAC_PARAMS)
model_sac.policy.to('cuda')

if if_using_sac:
  # set up logger
  tmp_path = RESULTS_DIR + '/sac'
  new_logger_sac = configure(tmp_path, ["stdout", "csv", "tensorboard"])
  # Set new logger
  model_sac.set_logger(new_logger_sac)

trained_sac = agent.train_model(model=model_sac, 
                             tb_log_name='sac',
                             total_timesteps=40000) if if_using_sac else None

{'batch_size': 128, 'buffer_size': 100000, 'learning_rate': 0.0001, 'learning_starts': 100, 'ent_coef': 'auto_0.1'}
Using cuda device
Logging to results/sac


------------------------------------
| time/              |             |
|    episodes        | 4           |
|    fps             | 31          |
|    time_elapsed    | 92          |
|    total_timesteps | 2912        |
| train/             |             |
|    actor_loss      | -176        |
|    critic_loss     | 142         |
|    ent_coef        | 0.0765      |
|    ent_coef_loss   | -1.15e+03   |
|    learning_rate   | 0.0001      |
|    n_updates       | 2811        |
|    reward          | -0.09952678 |
------------------------------------
day: 727, episode: 10
begin_total_asset: 1000000.00
end_total_asset: 1301585.01
total_reward: 301585.01
total_cost: 124418.37
total_trades: 129221
Sharpe: 0.493
-----------------------------------
| time/              |            |
|    episodes        | 8          |
|    fps             | 31         |
|    time_elapsed    | 182        |
|    total_timesteps | 5824       |
| train/             |            |
|    actor_loss      | -339     

In [23]:
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3 import SAC
from finrl.agents.stablebaselines3.models import DRLAgent
from finrl.meta.env_stock_trading.env_stocktrading import StockTradingEnv
import numpy as np
from sklearn.metrics import accuracy_score, roc_auc_score, f1_score

# 创建向量化环境
vec_env = make_vec_env(lambda: StockTradingEnv(df=trade, **env_kwargs), n_envs=1)

model_sac.load(RESULTS_DIR + '/sac')

# 重置环境
obs = vec_env.reset()
done = False
predictions = []
actuals = []

while not done:
    action, _states = model_sac.predict(obs, deterministic=True)
    obs, rewards, done, info = vec_env.step(action)
    predictions.append(action[0] if action.size == 1 else action)  # 根据你的模型输出调整
    actuals.append(info[0]['actual'])  # 从info字典中获取实际信息

# 转换为二进制标签
pred_labels = np.array(predictions) > 0
actual_labels = np.array(actuals) > 0

# 计算分类指标
accuracy = accuracy_score(actual_labels, pred_labels)
auc = roc_auc_score(actual_labels, pred_labels)
f1 = f1_score(actual_labels, pred_labels)

print("Accuracy:", accuracy)
print("AUC:", auc)
print("F1-Score:", f1)

IsADirectoryError: [Errno 21] Is a directory: 'results/sac'

In [18]:
print("==============Get Backtest Results===========")
now = datetime.datetime.now().strftime('%Y%m%d-%Hh%M')

perf_stats_all = backtest_stats(account_value=df_account_value)
perf_stats_all = pd.DataFrame(perf_stats_all)
perf_stats_all.to_csv("./"+RESULTS_DIR+"/perf_stats_all_"+now+'.csv')

==============Get Backtest Results===========
Annual return         -0.001131
Cumulative returns    -0.001086
Annual volatility      0.000988
Sharpe ratio          -1.149330
Calmar ratio          -1.041299
Stability              0.012397
Max drawdown          -0.001086
Omega ratio            0.000000
Sortino ratio         -1.148703
Skew                        NaN
Kurtosis                    NaN
Tail ratio                  NaN
Daily value at risk   -0.000129
dtype: float64


/home/autolab/miniconda3/envs/fintech/lib/python3.10/site-packages/empyrical/stats.py:1527: RuntimeWarning: invalid value encountered in scalar divide
  return np.abs(np.percentile(returns, 95)) / \
